In [ ]:
import os
import json
import time
import yaml
import urllib
from tqdm import tqdm

In [ ]:
date = '20200130'

In [ ]:
def load_config(path=None):
    if path is None:
        path = os.path.join(os.getcwd(), 'config.yml')
    with open(path, 'r', encoding='UTF-8') as yml:
        config = yaml.load(yml, Loader=yaml.FullLoader)
    return config

In [ ]:
config = load_config()

In [ ]:
files_url = "https://slack.com/api/files.list"
files_dict_keys = ["token", "count"]
files_dict = {key: config[key] for key in files_dict_keys}

In [ ]:
files_dict['page'] = 1
files = f"{files_url}?{urllib.parse.urlencode(files_dict)}"
files_result = json.loads(urllib.request.urlopen(files).read())
time.sleep(1)

In [ ]:
files_result['paging']

In [ ]:
file_name_urls = {}
for file in files_result['files']:
    f_name = file["name"]
    f_url = file['url_private']
    suffix_n = 0
    f_name_split = os.path.splitext(f_name)
    while f_name in file_name_urls.keys():
        f_name = f_name_split[0] + '_' + str(suffix_n) + f_name_split[1]
        suffix_n += 1
    file_name_urls.update({f_name: f_url})

In [ ]:
i = 0
auth_dict = {"Authorization": f"Bearer {config['token']}"}
for item in tqdm(file_name_urls.items()):
    url = item[1]
    file_name = item[0]
    file_path = f"downloaded/{date}/{file_name.replace('/', '')}"
    if os.path.exists(file_path):
        continue
    req = urllib.request.Request(url, headers=auth_dict, method="GET")
    try:
        with urllib.request.urlopen(req) as res:
            file_byte = res.read()
        with open(file_path, mode="wb") as f:
            f.write(file_byte)
    except urllib.error.HTTPError:
        print(f'Could not download {file_name}')
    time.sleep(1)